In [1]:
#!pip install --user -r requirements.txt

In [27]:
# check the device
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [22]:
import random
import time
import numpy as np
import sys
import os
# Set up TWINT config
import twint
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio

# stopword
import nltk
from nltk.corpus import stopwords
nltk.download("vader_lexicon")
nltk.download('stopwords')
sw_nltk = stopwords.words('english')

import re
text_cleaning_regex = "@S+|https?:S+|http?:S|[^A-Za-z0-9]+"

from keras.models import Sequential
from keras.layers import LSTM,Flatten,Embedding,Dense,Conv2D,MaxPooling2D,Conv1D


# sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# bert
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler,random_split
from transformers import BertModel,BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AdamW, get_linear_schedule_with_warmup


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/swu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/swu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nest_asyncio.apply()
c = twint.Config()
c.Search = "BTC"
c.Lang = "en"
c.Store_pandas = True
# c.Store_csv = True
# c.Output = "test.csv"
c.Limit = 1000
c.Pandas = True
twint.run.Search(c)

1493394126498406400 2022-02-14 17:18:00 -0800 <Dominik98205914>  https://t.co/g4Mg99UDL6  Free btc
1493394125424676864 2022-02-14 17:18:00 -0800 <EGalimidi9> @Bitcoin @BilgeEbiri Both of these things can be true for the reasons each of you same them. Odds are you’re both right.
1493394097457221632 2022-02-14 17:17:53 -0800 <Bobbybdck> Please I need to submit now! Any domme please message and rinse me 🥺🥺  Findom femdom finsub paypig findomaus btc bitcoin relapse beta simp goddess mistress domme beemit paypal
1493394095292567554 2022-02-14 17:17:52 -0800 <RahulCh80541472> @sumitkapoor16 @RBI Most of the Indian investors have 100 to 1000 $ in their wallets. But the way government is imposing tax on us, it seems they think every Indian crypto investor is a millionaire. This is not true. If it so, why don't government just buy 10,000 Btc for next 5 years n become rich.
1493394086040064001 2022-02-14 17:17:50 -0800 <WastedCrypto> @Croesus_BTC From 7 years ago. Shut up dude.
14933940743967621

1493393666391613443 2022-02-14 17:16:10 -0800 <NY1776> #btc #NFT  https://t.co/bz8UyxAsJL
1493393666089447424 2022-02-14 17:16:10 -0800 <0xSifu> @TheRedPen87 @nikigranger Investigations say that most of your btc from Q was traded away, the rest was seized. Are you not paying attention to your own case ?    https://t.co/izpazYXV6y
1493393656992002051 2022-02-14 17:16:08 -0800 <cryptocharts> Thoughts on $BTC #Bitcoin! See CryptoAfterDark's idea on TradingView below.  https://t.co/dSAEIizYfL
1493393654374969346 2022-02-14 17:16:07 -0800 <MarketChartsApp> #XBTUSD - New Technical Analysis   Chart on TradingView  https://t.co/ecR28JQAUJ $BTC $ETH #Bitcoin #CryptoCurrency #Trading #TechnicalAnalysis  https://t.co/SEcWEoBDoK
1493393642366533633 2022-02-14 17:16:04 -0800 <santa_cruzz02> A los que tanto critican #USDT, si quieren les paso la dirección de mi wallet de #USDT en #Binance y se deshacen de ellos 🤠 #Crypto #BTC #Coinbase #Bitcoin
1493393632644194306 2022-02-14 17:16:02 -0800 <Nino_P11

1493393329710534659 2022-02-14 17:14:50 -0800 <8bitslong> Like coinbase has any track record being able to stay afloat with any real traffic. #crypto You'd think that be a requirement. #btc  https://t.co/AV8k6peKJM
1493393324991864832 2022-02-14 17:14:49 -0800 <RahulCh80541472> @KB10051978 @sumitkapoor16 @RBI Most of the Indian investors have 100 to 1000 $ in their wallets. But the way government is imposing tax on us, it seems they think every Indian crypto investor is a millionaire. This is not true. If it so, why don't government just buy 10,000 Btc for next 5 years n become rich.,
1493393324190822401 2022-02-14 17:14:49 -0800 <Shanaz3197> #11MINUTES #BTC #CRYPTO  https://t.co/A5RbL8opyo
1493393323792416768 2022-02-14 17:14:48 -0800 <media_diamante> EQONEX to develop product suite: announced BTC dated futures    https://t.co/RbdnsKEjcX    #Diamante #Blockchain #DiamCircle
1493393318104862721 2022-02-14 17:14:47 -0800 <NeoPassport> THIS is why we #Bitcoin ⤵️ #BTC = Unconfiscatable st

1493393007143387136 2022-02-14 17:13:33 -0800 <parman_the> @backpacker_btc @francispouliot_ Loud noises!
1493393003674628098 2022-02-14 17:13:32 -0800 <bonus_binance> こちらの招待コードC61V3I6Cからバイナンスへ登録して頂くと取引額から手数料10%がキャッシュバックされます。  #Binance  #バイナンス #BTC  https://t.co/n8SsP1fjcj
1493392973182218246 2022-02-14 17:13:25 -0800 <JuniorsCrypto> Is the total market cap chart look bullish to anyone else?  Cup &amp; Handle with a falling wedge on the 4 HR  Odds of upside are in pretty high favor 👀  $VRA $MTV $VXV $LCX $QNT $RMRK $KDA $MNW $XPR $BTC #Bitcoin #altcoin #cryptocurrency #CryptoNews #chartpatterns $ETH $FTM  https://t.co/aKNxWkAlkw
1493392962784530432 2022-02-14 17:13:22 -0800 <btc_blockbot> $BTC Latest Block Info: Block 723344 holds 954 transactions for a total of 5,234.13307 #Bitcoin including Block Reward. Block Size 819,890 Bytes
1493392953313611777 2022-02-14 17:13:20 -0800 <Pomegranate613> #btc 📈 The executive cautioned companies that don't recognize the advances of blockchain techno

1493392519714934784 2022-02-14 17:11:37 -0800 <Will_Leas> 📰News -@LensProtocol unveiled -@terra_money x @Nationals  -DOJ recovers 94k BTC from 2016 Bitfinex hack -@opensea launches venture arm -@SuperBowl crypto ads: @FTX_Official w/ Larry David for the content &gt; @coinbase's QR in my books (halftime show was also🔥)
1493392517152518146 2022-02-14 17:11:36 -0800 <ragnaroks751> @TechDev_52 spam short leverage trades im sure BTC will pump hard..
1493392512786087943 2022-02-14 17:11:35 -0800 <srdrsny> hodlnaut has the best #BTC #saving rate of 6.7% on  https://t.co/utTfUWtG7n🔥. Our rates update in real-time, so be sure to keep an eye on it #DeFi 👀             https://t.co/QsiFERMnMd
1493392510764429318 2022-02-14 17:11:35 -0800 <amarquez2002> @AltCryptoGems I am glad to have met BTC years ago, and of course to have run into NWC these days. They will certainly mark a point in my life
1493392498143678464 2022-02-14 17:11:32 -0800 <supernova01_s> @yatrmci_85 burdanda gidebilir onayada geleb

...

1493387967716544515 2022-02-14 16:53:31 -0800 <cryptoquanimous> #bitcoin an orb of enlightenment. #btc a frequency of hope. A paradoxical aura of value.
1493387954324221955 2022-02-14 16:53:28 -0800 <mbzps> ما وافت القمه الثانية للاعلى من الي قبلها ، ارتداد #BTC  https://t.co/AF6uFoCFVt
1493387942399979520 2022-02-14 16:53:25 -0800 <Santy8513> Trending on #LunarCrush:  "Elon Musk Hints SpaceX Could Soon Accept Dogecoin ($DOGE) Payments" via @CryptoGlobeInfo  Top coin mentions $btc $doge   https://t.co/KvZ3eZENP0
1493387921008717825 2022-02-14 16:53:20 -0800 <uMY9jNAlbgsB5YW> 40500タッチして全戻しが綺麗かー🚀  #BTC
1493387917397532675 2022-02-14 16:53:19 -0800 <cripnaldo> Pensando en acumular algún buen proyecto?? #AVAX esta cerca del punto 5  Fibo... #BTC  https://t.co/cucZGyp6pk
1493387911546482691 2022-02-14 16:53:18 -0800 <additionalrules> News about Metaverse  Title：Why Is the Metaverse the New Buzz in the Crypto Space? - Yahoo Finance    https://t.co/T2l1Sn7UN7  #Metaverse #meta #crypto #BTC #b

1493387506628407297 2022-02-14 16:51:42 -0800 <printcryptoUSA> ❤️‍🔥New US inventory alert! Antminer S19j Pro (104th) $BTC miners in stock and ready to ship. Grab yours at  https://t.co/HDKWl2gFZ4 ❤️‍🔥
1493387493407969281 2022-02-14 16:51:38 -0800 <momcashtags> 2012 you missed $BTC 2013 you missed $DOGE 2014 you missed $XRP 2015 you missed $ETH  2016 you missed $ADA  2017 you missed $BNB  2018 you missed $LINK  2019 you missed $DOT  2020 you missed $SHIB  2021 you missed $SOL   In 2022 don’t miss $MContent #MContent  @MContent_ 💎🚀
1493387491495141380 2022-02-14 16:51:38 -0800 <ValenzuelaPost> Peter Schiff Predicts Bitcoin (BTC) at $10,000  https://t.co/VKifRstI1i
1493387490714996741 2022-02-14 16:51:38 -0800 <cryptolover103> #11MINUTES #BTC #CRYPTO
1493387480602939393 2022-02-14 16:51:35 -0800 <Knownasjk1> @IamCryptoWolf Btc sideways for two years. The crypto asset class as a whole is becoming lame. Diminishing returns and super extended cycles.
1493387474030059522 2022-02-14 16:51:34 -

1493387015886356481 2022-02-14 16:49:45 -0800 <Observadorsv503> @eliel_scary312 @nayibbukele El BTC trae inversores en BTC, hay muchos bitcoiners "desamparados" que no encuentran dónde invertir, buscan un lugar con libertad económica y con pocos impuestos.  El mundo está cambiando, la pregunta no es si el BTC es viable, la pregunta es ¿Cuándo? Y los primeros ganan.
1493386993178402820 2022-02-14 16:49:39 -0800 <goldfx52> 暗号通貨の取引所は Bybitがおすすめ！ 日本の販売所は手数料が高いです 特にハイリターンなFXをされる方は是非 今ならこちらから口座開設で$10相当のBTCが貰えます  https://t.co/QfaKy1Og3H バイナンス bitbank BTC ビットコイン コインチェック
1493386993157296130 2022-02-14 16:49:39 -0800 <EyeofVisceral> @nayibbukele I'm qualified to operate geothermal power plants and have $BTC and #crypto on my resume.
1493386985452765184 2022-02-14 16:49:37 -0800 <Santy8513> Trending on #LunarCrush:  "Couple fights detention in alleged laundering of $3.6 billion in hacked bitcoin funds" via @washingtonpost  Top coin mentions $btc   https://t.co/krF9FwZe3F
1493386971690975232 2022-

1493386452591423489 2022-02-14 16:47:30 -0800 <scorvorite> @TorontoStar Try stopping #BTC. The people will prevail. You have no power when the masses rise in opposition.
1493386450964193281 2022-02-14 16:47:30 -0800 <LoloyMana> Trending on #LunarCrush:  "Crypto companies spent millions on Super Bowl ads, but it didn’t lift bitcoin price Monday" via @MarketWatch  Top coin mentions $btc   https://t.co/7uBIxvGISC
1493386436409585664 2022-02-14 16:47:26 -0800 <LoiselNicole> @TheGuySwann Why are those who are passionate about crypto and BTC so aligned on convoys planned and managed through fake social media mills in Asia?  It's a sad day when the crypto community rallies behind this.
1493386427366735874 2022-02-14 16:47:24 -0800 <ben07338053> @LadyofCrypto1 How is that good for BTC , the banks will run it down to $0
1493386426301440003 2022-02-14 16:47:24 -0800 <LuisBecerra18> @100trillionUSD BTC chart after 14 years, is showing 2 diferents realities.  1.  ATH  2. Higher lows.   This number

1493386018648735749 2022-02-14 16:45:47 -0800 <CryptoKubrick98> This Rich but Degen is really Genius!  ✅💥 @Altsteinn 💴  I made x4 to my funds thnx to his recommendation to buy $CEEK  Subscribe and see for yourself  $vet $sand $cake $icp $kai $theta $sfund $egld $vra $pols $inj $caps $ulti $bnb $soul $rari $gfn $cspr $btc $ckb $wrx
1493386018422079489 2022-02-14 16:45:47 -0800 <LoiselNicole> Why are those who are passionate about crypto and BTC so aligned on convoys planned and managed through fake social media mills in Asia?  It's a sad day when the crypto community rallies behind this.
1493386004228718595 2022-02-14 16:45:43 -0800 <JCnum1theGame> Faith! Faith what? 🤣 That's why I sell the US DOLLAR for BTC! 👽 wake up!
1493386003859394564 2022-02-14 16:45:43 -0800 <crypto_advocate> IMO $btc takes another stab at 46k from here. On flip side if 41k gives, expecting 38.5k.  https://t.co/9sK3IWGK9x
1493385999761743873 2022-02-14 16:45:42 -0800 <MarketChartsApp> #GRTUSD - New Technical Anal

In [3]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
df = twint_to_pandas(["date", "username","tweet", "hashtags", "nreplies", "nretweets","nlikes"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       1011 non-null   object
 1   username   1011 non-null   object
 2   tweet      1011 non-null   object
 3   hashtags   1011 non-null   object
 4   nreplies   1011 non-null   int64 
 5   nretweets  1011 non-null   int64 
 6   nlikes     1011 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 55.4+ KB


## Sentiment analysis

### 1. Preprocessing

In [4]:
def text_preprocessing(tweet):
    tweet = re.sub(text_cleaning_regex, ' ', str(tweet).lower()).strip()
    res=[]
    # Lowercase
    tweet = tweet.lower()
    # Remove single letter words
    # tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    # Remove stopword
    tweet = ' '.join([word for word in tweet.split() if word.lower() not in sw_nltk])
    # remove the word after @ OR #
    for i in tweet.split():
        if i.startswith("@") or i.startswith("#"):
            continue
        else:
            res.append(i)
    return ' '.join(res)

In [5]:
df['tweet']=df['tweet'].apply(lambda x: text_preprocessing(x))

<ipython-input-5-b753ad286cc5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet']=df['tweet'].apply(lambda x: text_preprocessing(x))


In [6]:
df.head()

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-02-14 17:18:00,Dominik98205914,https co g4mg99udl6 free btc,[],0,0,0
1,2022-02-14 17:18:00,EGalimidi9,bitcoin bilgeebiri things true reasons odds right,[],0,0,0
2,2022-02-14 17:17:53,Bobbybdck,please need submit domme please message rinse ...,[],0,0,0
3,2022-02-14 17:17:52,RahulCh80541472,sumitkapoor16 rbi indian investors 100 1000 wa...,[],0,0,0
4,2022-02-14 17:17:50,WastedCrypto,croesus btc 7 years ago shut dude,[],0,0,0


### 2. prepare the data

Generate Labels using an Unsupervised method


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    if scores["neg"] > scores["pos"]:
        return 0
    return 1

In [8]:
df["vader_result"] = df["tweet"].apply(lambda x: vader_sentiment_result(x))
df.head()

<ipython-input-8-cb268a706877>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["vader_result"] = df["tweet"].apply(lambda x: vader_sentiment_result(x))


,date,username,tweet,hashtags,nreplies,nretweets,nlikes,vader_result
0,2022-02-14 17:18:00,Dominik98205914,https co g4mg99udl6 free btc,[],0,0,0,1
1,2022-02-14 17:18:00,EGalimidi9,bitcoin bilgeebiri things true reasons odds right,[],0,0,0,1
2,2022-02-14 17:17:53,Bobbybdck,please need submit domme please message rinse ...,[],0,0,0,1
3,2022-02-14 17:17:52,RahulCh80541472,sumitkapoor16 rbi indian investors 100 1000 wa...,[],0,0,0,1
4,2022-02-14 17:17:50,WastedCrypto,croesus btc 7 years ago shut dude,[],0,0,0,1


In [14]:
X = df.tweet.values
y = df.vader_result.values
X_train, X_val, y_train, y_val =train_test_split(X, y, test_size=0.1, random_state=2)

In [15]:
# Create a function to tokenize the input for encoder 
# Load bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=sent,  
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=280,                  # maximum twitter lenght 280
            pad_to_max_length=True,         
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      
            truncation=True
            )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
     
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    return input_ids, attention_masks

In [17]:
# create input 
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, # The training samples.
                              sampler=train_sampler,   # Select batches randomly
                              batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, 
                            sampler=val_sampler,  # Pull out batches sequentially
                            batch_size=batch_size)

### 3. model training


#### 3.1 create BertClassifier

In [32]:
class BertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        self.bert =  BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)
        
        return logits

#### 3.2 Optimizer & Learning Rate Scheduler

In [18]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [40]:
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    bert_classifier = BertClassifier()


#     #Tell PyTorch to run the model on GPU
#     bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(model.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

#### 3.3 Training Loop

In [41]:
# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
    
def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # training, Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        
        # Evaluation
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance on val set
            val_loss, val_accuracy = evaluate(model, val_dataloader)
            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete")
    
    
def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance on val set
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### 3.4. Prediction

In [79]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [91]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)
# Get predictions from the probabilities
threshold = 0.5
preds = np.where(probs > threshold, 1, 0)
# Number of tweets predicted non-negative
print("Number of tweets predicted non-negative: ", preds.sum())

Number of tweets predicted non-negative:  100
